![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [145]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [146]:
cc_apps = cc_apps.replace("?", np.nan)

In [147]:
cc_apps_copy = cc_apps.copy()

In [148]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    object 
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [149]:
# https://stackoverflow.com/questions/67514298/how-to-extract-values-from-pandas-value-counts-results
max(cc_apps[3].value_counts().to_dict().values())

519

In [150]:
for col in cc_apps.columns:
    if cc_apps[col].dtype == "object":
        cc_apps[col] = cc_apps[col].fillna(max(cc_apps[3].value_counts().to_dict().values()))
        
    else:
        cc_apps[col] = cc_apps[col].fillna(cc_apps[col].mean())

In [151]:
# One-hot encoding
cc_apps = pd.get_dummies(cc_apps, drop_first=True)
cc_apps

,2,7,10,12,0_a,0_b,1_13.75,1_15.17,1_15.75,1_15.83,1_15.92,1_16.00,1_16.08,1_16.17,1_16.25,1_16.33,1_16.50,1_16.92,1_17.08,1_17.25,1_17.33,1_17.42,1_17.50,1_17.58,1_17.67,1_17.83,1_17.92,1_18.00,1_18.08,1_18.17,1_18.25,1_18.33,1_18.42,1_18.50,1_18.58,1_18.67,1_18.75,1_18.83,1_18.92,1_19.00,...,1_69.50,1_71.58,1_73.42,1_74.83,1_76.75,1_80.25,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s,13_-
0,0.000,1.25,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
1,4.460,3.04,6,560,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
2,0.500,1.50,0,824,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,1.540,3.75,5,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
4,5.625,1.71,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,10.085,1.25,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
686,0.750,2.00,2,394,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
687,13.500,2.00,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
688,0.205,0.04,0,750,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [152]:
features = cc_apps.drop(cc_apps.columns[-1], axis=1).values

# https://stackoverflow.com/questions/40144769/how-to-select-the-last-column-of-dataframe
target = cc_apps.iloc[:, -1].values


we can also get features using this:<br>
`cc_apps.iloc[:, :-1].values`

In [153]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.3, random_state=101)

In [154]:
train_features.shape, test_features.shape

((483, 388), (207, 388))

In [155]:
scaler = StandardScaler()

In [156]:
rescaled_train_features = scaler.fit_transform(train_features)
rescaled_test_features = scaler.fit_transform(test_features)

In [157]:
log_model = LogisticRegression()
log_model

LogisticRegression()

In [158]:
log_model.fit(rescaled_train_features, train_target)

LogisticRegression()

In [159]:
test_target_preds = log_model.predict(rescaled_test_features)

In [160]:
np.mean(test_target_preds == test_target)

0.7729468599033816

In [161]:
train_target_preds = log_model.predict(rescaled_train_features)

In [162]:
np.mean(train_target_preds == train_target)

0.9979296066252588

In [163]:
print("Training confusion matrix:")

confusion_matrix(train_target, train_target_preds)

Training confusion matrix:


array([[218,   0],
       [  1, 264]])

In [164]:
print("Testing confusion matrix:")

confusion_matrix(test_target, test_target_preds)

Testing confusion matrix:


array([[66, 23],
       [24, 94]])

In [165]:
tol = [0.1, 0.01, 0.001, 0.0001]
max_iter = [50, 100, 150, 200]

In [166]:
grid_model = GridSearchCV(estimator=log_model, param_grid=params, cv=5)

In [167]:
grid_model_result = grid_model.fit(rescaled_train_features, train_target)

In [168]:
best_train_score, best_train_params = grid_model_result.best_score_, grid_model_result.best_params_

In [169]:
best_model = grid_model_result.best_estimator_

test_target_preds_best = best_model.predict(rescaled_test_features)

# Evaluate the model on the test set
test_confusion_matrix = confusion_matrix(test_target, test_target_preds_best)
test_accuracy = accuracy_score(test_target, test_target_preds_best)

In [170]:
print(test_confusion_matrix, test_accuracy)

[[66 23]
 [24 94]] 0.7729468599033816


In [171]:
best_score = best_model.score(rescaled_test_features, test_target)

In [172]:
print(f"Accuracy of logistic regression classifier: {best_score}")

Accuracy of logistic regression classifier: 0.7729468599033816
